***Quantum Random Walk***

Quantum walks are quantum analog of classical random walk. Quantum advantage comes from the (i) superpostion of the quantum states (ii) reversible unitary evolution (iii) collapse of the wave-function on one of the states. The algorithm has mainly two parts, a function that forms the probability vector for the movement along the given geometry. We define the function as ***W_operator*** and other function swap/shift operator which performs actual movement of the walker by applying swap function. Here we represent that function with ***S_operator*** 

In [8]:

from classiq import *

size = 4

@qfunc
def prepare_minus(x: QBit):
    '''Creates a function |-> which is equal 
    superpostion of |0> and |1> states.
    '''
    X(x)
    H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
    '''Applies phase kickback operation on the auxiliary qubit'''
    aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
    '''Applies diffuser_oracle function to auxiliary qubit which is
    prepared in |-> state.
    '''
    aux = QNum('aux')
    allocate(1,aux)
    within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    
    prob = [0]* 2**size
    if i==0:
        prob[i+1] = 1.0 #Probability for moving to right of the first node in the line
    elif i==((2**size)-1):
        prob[i-1] = 1.0 #Probability for moving to left of the last node in the line
    else:
        prob[i-1] = 0.5 #Probability of moving to left of the node
        prob[i+1] = 0.5 #Probability of moving to right of the node
    print(f'State={i}, prob vec ={prob}')
    
    
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))
    

@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    '''Applies W_operators to all the vertices'''
    for i in range(2**size):
        W_iteration(i,vertices,adjacent_vertices)


@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
    '''Check which vertices are adjacent to each other by comparing the two nodes.
    If the difference is +1 / -1, it should be adjacent in a line of nodes. 
    '''
    res |= (((vertices-adjacent_vertices) == 1) | ((vertices-adjacent_vertices) == -1))


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
    '''Swaps the two vertices'''
    repeat(count= x.len,
        iteration= lambda i: SWAP(x[i],y[i]))
  

@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    '''Checks if the two vertices are adjacent and if they are adjacent, it applies the bitwise_swap 
    function to swap the two nodes, thus performing actual movement of quantum walker.'''
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
    '''Main function that puts everything together to perform quantum random walk'''

    allocate(size,vertices)
    hadamard_transform(vertices)
    allocate(size,adjacent_vertices)
    
    W_operator(vertices,adjacent_vertices)
    S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)



State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0

***Output circuit visualization***

Here is the full circuit view, where you see the whole circuit is implemented using 37 qubits. 
The Hadamard operator creates an equal superposition of the 4 qubits giving us a vector space of 16 (our desired number of nodes).

![alt text](full_circuit_quantum_walk.jpg "full_circuit")

We further zoom in on each of the functions. When we zoom in on the W_operator, we see the state_preparation, zero_diffuser (which formulates the probability vector), and inverse_state_prepapation. These functions/operators are applied to each node. 

![alt text](W_operator_quantum_walk.png "W_operator")

Here we zoom in on the shift/swap operator. Edge_oracle checks if the two nodes are connected by and edge (if they are adjacent). If they are connected by an edge, the bitwise_swap operator swaps the two nodes, mimicking the quantum walk (the actual movement of the walker). It is a controlled operation, and it is only performed if the edge_oracle finds the two nodes adjacent. 

![alt text](S_operator_quantum_walk.png "S_operator")